In [0]:
from pandas.io.json import json_normalize
import psycopg2
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import gensim, logging
from sklearn.cluster import DBSCAN
from sklearn import preprocessing, metrics
from collections import defaultdict

### Step 0: Read RSS to dataframe, transform dates, create 24hr buckets and 4-day buckets

In [1]:
from pandas.io.json import json_normalize
import psycopg2
import pandas as pd
import datetime

### Step 0: read RSS to dataframe, transform dates, create 24hr buckets and 4-day buckets
def read_rss():
    database = "rssfeed"
    hostname="rssfeed.cjgj2uy1bapa.us-east-1.rds.amazonaws.com"
    port="5432" 
    userid="postgres"
    passwrd=""
    conn_string = "host="+hostname+" port="+port+" dbname="+database+" user="+userid+" password="+passwrd
    conn = psycopg2.connect(conn_string)
    conn.autocommit=True
    cursor = conn.cursor()
    sqlSelect = "SELECT * FROM rss_entities_with_pub";
    cursor.execute(sqlSelect);
    rows = cursor.fetchall();
    return rows

def read_publishers():
    publishers = pd.read_json('data/rss_feed.json', typ='series')
    publishers = publishers.to_dict()
    pub = {v: k for k, v in publishers.items()}
    return pub

def make_rssdf():
    rows = read_rss()
    pub = read_publishers()
    rssdf = pd.DataFrame(rows)
    rssdf.columns = ['publisher_code','entity','publish_time']
    rssdf["publisher"] = rssdf["publisher_code"].map(pub)  
    return rssdf

def add_24hr_bucket():
    rssdf = make_rssdf()
    rssdf['24hour_bucket'] = pd.to_datetime(
        rssdf['publish_time'],errors='coerce',format = '%Y-%m-%dT%H:%M:%S+00:00',\
        infer_datetime_format = True, cache = True)
    return rssdf

def group_entities_by_bucket():
    rssdf = add_24hr_bucket()
    rssdf['dates']=rssdf['24hour_bucket'].dt.date
    newdf = pd.DataFrame(rssdf.groupby(['dates'])['entity']\
                      .apply(", ".join)).reset_index().sort_values(by=['dates'])
    newdf['dates'] = pd.to_datetime(newdf['dates'])
    newdf = newdf[newdf['dates']<datetime.date(2020,3,19)] 
    return newdf

def CountFrequency(my_list): 
    '''
    Given a list of entities, return a dictionary of frequencies
    '''
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

def aggregate_entities_4_days():
    '''
    Create another column in the given dataframe,
    combine 4 consecutive days (slicing window approach),
    store it in the column "entities_4_days" as a dictionary (key = entity, val = frequency)
    '''
    mydf = group_entities_by_bucket()
    mydf_2007_plus = mydf[(mydf['dates'] >= pd.Timestamp(datetime.date(2007,1,1)))].reset_index()
    lst = []
    for i in range(len(mydf_2007_plus)-4):
        mystr = mydf_2007_plus.iloc[i,2]+ ", " + mydf_2007_plus.iloc[i+1,2] + ", "+\
                   mydf_2007_plus.iloc[i+2,2] + ", " + mydf_2007_plus.iloc[i+3,2]
        mylist = list(mystr.split(", ")) 
        mydict = CountFrequency(mylist)
        lst.append(mydict)

    mydf_2007_plus['entities_4_days'] = lst + [np.nan] * 4 
    return mydf_2007_plus, lst

def refine_entity_list():
    '''
    Create a separate column of entities with frequency > 1
    '''
    lst2 = []
    df_window, lst = aggregate_entities_4_days()
    for i in range(len(lst)):
        d = lst[i]
        lst2.append({key: d[key] for key in d if d[key] > 1})
    df_window['refined_entities_4_days'] = lst2 + [np.nan] * 4 
    del df_window['index']
    return df_window, lst2

df_window, lst2 = refine_entity_list()
df_window.to_csv('data/df_window.csv', index = False)
df_window.head()

ModuleNotFoundError: No module named 'psycopg2'

In [0]:
### Subset the first 10 rows for quick demonstration
df_sub = df_window.iloc[0:50,:]
df_sub

,dates,entity,entities_4_days,refined_entities_4_days
0,2007-01-01,"Episode, iTunes, New Year, Radio Detective Sto...","{'Episode': 1, 'iTunes': 1, 'New Year': 2, 'Ra...","{'New Year': 2, 'German': 2, 'New Year Resolut..."
1,2007-01-02,"Prager, Who Really Cares: The suprising, Denni...","{'Prager': 1, 'Who Really Cares: The suprising...","{'Italian': 2, 'IndieFeed Alternative Modern R..."
2,2007-01-03,"Idiotchild - Mouth Watering, IndieFeed Alterna...","{'Idiotchild - Mouth Watering': 1, 'IndieFeed ...","{'IndieFeed Alternative Modern Rock': 2, 'Indi..."
3,2007-01-04,"Speeding Bullets, DC, Raging Bullets of, Jim, ...","{'Speeding Bullets': 1, 'DC': 1, 'Raging Bulle...","{'Sarah Dinan': 2, 'Coyote Run': 2, 'Marc Gunn..."
4,2007-01-05,"Irish & Celtic Music Podcast, Celtic, Sarah Di...","{'Irish & Celtic Music Podcast': 1, 'Celtic': ...","{'Sarah Dinan': 2, 'Coyote Run': 2, 'Marc Gunn..."
5,2007-01-06,"Q & A, Pixie, Ann Druyan, Carl Sagan, Ann Druy...","{'Q & A': 1, 'Pixie': 1, 'Ann Druyan': 2, 'Car...","{'Ann Druyan': 2, 'Carl Sagan': 4, 'Italian': ..."
6,2007-01-07,"Italian, Zencast 86 - Right Livelihood, Gil Fr...","{'Italian': 6, 'Zencast 86 - Right Livelihood'...","{'Italian': 6, 'Gil Fronsdal': 2, 'Matt White ..."
7,2007-01-08,"Photoshop CS3 Way, Photoshop CS3 Public Beta, ...","{'Photoshop CS3 Way': 1, 'Photoshop CS3 Public...","{'North': 2, 'IndieFeed Alternative Modern Roc..."
8,2007-01-09,"Italian, Italian, Apple, Steve Jobs, Macworld ...","{'Italian': 8, 'Apple': 1, 'Steve Jobs': 1, 'M...","{'Italian': 8, 'Ben': 3, 'Spanish': 3, 'Guava ..."
9,2007-01-10,"English, Unnateral Helpers - Gettin', IndieFee...","{'English': 1, 'Unnateral Helpers - Gettin'': ...","{'Guava Duff': 2, 'Italian': 7, 'Spanish': 2, ..."


### Step 1: Detect Events

In [2]:
### Step 1: Detect Events
def create_design_matrix(row, sentences, model):
    a = sentences[row]
    designX = []
    for i in range(len(a)):
        try:
            designX.append(model[a[i]])
        except KeyError:
            print(a[i])
            pass
    return designX, a

def standardize_design_matrix(designX):
    standardX = preprocessing.StandardScaler().fit_transform(designX)
    return standardX

def argmax(s): 
    f = lambda i: s[i]
    return max(range(len(s)), key=f)

def best_eps(eps, standardX):
    s = []
    for i in range(len(eps)):
        e = eps[i]
        dbscan = DBSCAN(metric='cosine',eps=e, min_samples=2) 
        clusters = dbscan.fit_predict(standardX)
        nclus = len(set(clusters))
        if nclus < 2:
            s.append(0)
            pass
        else:
            s.append(metrics.silhouette_score(standardX, clusters))
    best = eps[argmax(s)]
    return best

def get_sentences_2(df_window):
    L = df_window['refined_entities_4_days'].dropna()
    sentences2 = []
    for i in range(len(L)):
        curr = list(L[i].keys())
        sentences2.append(curr)
    return sentences2

def get_sentences(df_window):
    m = [df_window['entity'][i] for i in range(len(df_window['entity']))]
    sentences = []
    for item in m:
        sentences.append(item.split(", "))
    return sentences

def train_word2vec_model(sentences):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = gensim.models.Word2Vec(min_count=1, workers=4)
    model.build_vocab(sentences, progress_per=200000)
    model.train(sentences, epochs=model.epochs, total_examples = model.corpus_count)
    #model.save('rssword2vec.model')
    #model = gensim.models.Word2Vec.load('rssword2vec.model')
    return model

def dbscan_cluster(standardX, a):
    '''
    Given a standardized design matrix, and a list of entities (a)
    Finds the best epsilon for DBSCAN, fit and predict with the best epsilon
    and return the grouping result in a dictionary
    if there's more than one group, also return the silhouette score
    '''
    # find best epsilon
    eps = [0.01,0.05,0.1,0.2,0.3,0.5,0.7,1,1.5,2,3,5,7,10]
    best = best_eps(eps, standardX)
    print(best)
    # train the model on the best epsilon
    dbscan = DBSCAN(metric='cosine',eps=best, min_samples=2) 
    clusters = dbscan.fit_predict(standardX)
    # put the grouping info in a dictionary
    testdict = dict(zip(a, clusters))
    res = defaultdict(list) 
    for key, val in sorted(testdict.items()): 
        res[val].append(key)
    # check number of groups and calculate silhouette scores
    nclus = len(set(clusters))
    if nclus > 1:
        sil = metrics.silhouette_score(standardX, clusters)
        return dict(res), sil
    else: 
        return dict(res), 0
    
def aggregate(df_window, nrows):
    '''
    Takes in a dataframe with grouped 24-hour buckets and 4-day buckets
    Trains the word2vec model on all entities
    For each 4-day time bucket, trains a DBSCAN model and detect events
    Evaluates each DBSCAN model with silhouette scores
    Returns the same dataframe with two added columns: groups (a dictionary) and silhouettes
    '''
    df_temp = df_window.iloc[0:nrows,:]
    sentences2 = get_sentences_2(df_window)
    sentences = get_sentences(df_window)
    print("sentences done")
    model = train_word2vec_model(sentences) # No need to train for each iteration!
    print("model done")
    groups = []
    silhouettes = []
    for i in range(nrows):  #len(sentences)
        row = i
        designX, a = create_design_matrix(row, sentences, model)
        standardX = standardize_design_matrix(designX)
        group, silhouette = dbscan_cluster(standardX, a)
        groups.append([group])
        silhouettes.append(silhouette)
    df_temp['groups'] = groups
    df_temp['silhouettes'] = silhouettes
    return df_temp
    #return groups, silhouettes

df_sub_clustered = aggregate(df_window, 50)
df_sub_clustered 

NameError: name 'df_window' is not defined

### Step 2: Determine event duration

In [0]:
def percent_overlap(a,b): 
    '''
    Takes in two lists of strings, return three floating numbers 
    in terms of percentage overlaps
    '''
    setA = set(a)
    setB = set(b)
    overlap = setA & setB
    universe = setA | setB
    result1 = float(len(overlap)) / len(setA) * 100
    result2 = float(len(overlap)) / len(setB) * 100
    result3 = float(len(overlap)) / len(universe) * 100
    return result1,result2,result3
    
def expand_groups_to_event_columns(df, col='groups'):
    '''
    Takes the dataframe from the output of "aggregate",
    Expands the "groups" column into separate columns, each labeled as an event
    '''
    to_dict = lambda x: x[0]
    df2 = json_normalize(df[col].apply(to_dict))
    wide_df = pd.concat([df, df2], axis=1)
    return wide_df

test = expand_groups_to_event_columns(df_sub_clustered)
test

,dates,entity,entities_4_days,refined_entities_4_days,groups,silhouettes,0,-1,2,1,3,6,4,5
0,2007-01-01,"Episode, iTunes, New Year, Radio Detective Sto...","{'Episode': 1, 'iTunes': 1, 'New Year': 2, 'Ra...","{'New Year': 2, 'German': 2, 'New Year Resolut...","[{0: ['- Murder Clinic', 'Anne Farnsworth', 'A...",0.630648,"[- Murder Clinic, Anne Farnsworth, Atari, Auld...","[CJ, Review, WindowtotheMagic, iTunes]","[Dan, Greg, Microsoft, Nintendo, Phil, Sony]","[German, New Year]",NaN,NaN,NaN,NaN
1,2007-01-02,"Prager, Who Really Cares: The suprising, Denni...","{'Prager': 1, 'Who Really Cares: The suprising...","{'Italian': 2, 'IndieFeed Alternative Modern R...","[{0: ['8MB', 'Arthur Brooks', 'Dennis Pragers'...",0.795786,"[8MB, Arthur Brooks, Dennis Pragers, New Year ...",[Dennis Prager],NaN,[Italian],NaN,NaN,NaN,NaN
2,2007-01-03,"Idiotchild - Mouth Watering, IndieFeed Alterna...","{'Idiotchild - Mouth Watering': 1, 'IndieFeed ...","{'IndieFeed Alternative Modern Rock': 2, 'Indi...","[{0: ['Caltech', 'Caltech Lecture Series', 'Ch...",0.726953,"[Caltech, Caltech Lecture Series, Chloe Day, C...","[Christian, New York Times]",[Sam Harris],"[IndieFeed Alternative Modern Rock, IndieFeed ...",[Spanish],NaN,NaN,NaN
3,2007-01-04,"Speeding Bullets, DC, Raging Bullets of, Jim, ...","{'Speeding Bullets': 1, 'DC': 1, 'Raging Bulle...","{'Sarah Dinan': 2, 'Coyote Run': 2, 'Marc Gunn...","[{0: ['0:00 Opening', '1:17:47 Manhunter Chall...",0.791652,"[0:00 Opening, 1:17:47 Manhunter Challenge, Br...",NaN,NaN,"[DC, Jim, Sean]",NaN,NaN,NaN,NaN
4,2007-01-05,"Irish & Celtic Music Podcast, Celtic, Sarah Di...","{'Irish & Celtic Music Podcast': 1, 'Celtic': ...","{'Sarah Dinan': 2, 'Coyote Run': 2, 'Marc Gunn...","[{1: ['1:00:32 Peggy', '43:04 Weave', '8:04 Ma...",0.659777,"[AP, AZ, American, Ben, Celtfather, Celtic, Ce...",NaN,NaN,"[1:00:32 Peggy, 43:04 Weave, 8:04 Mairi Mac, A...",NaN,NaN,NaN,NaN
5,2007-01-06,"Q & A, Pixie, Ann Druyan, Carl Sagan, Ann Druy...","{'Q & A': 1, 'Pixie': 1, 'Ann Druyan': 2, 'Car...","{'Ann Druyan': 2, 'Carl Sagan': 4, 'Italian': ...","[{1: ['A Famous Broken Heart', 'Ann Druyan', '...",0.640159,"[D.J. Grothe, NASA, New York Times, Q & A]",NaN,NaN,"[A Famous Broken Heart, Ann Druyan, Carl Sagan...",NaN,NaN,NaN,NaN
6,2007-01-07,"Italian, Zencast 86 - Right Livelihood, Gil Fr...","{'Italian': 6, 'Zencast 86 - Right Livelihood'...","{'Italian': 6, 'Gil Fronsdal': 2, 'Matt White ...","[{0: ['American', 'Dave', 'European Union', 'I...",0.624332,"[American, Dave, European Union, Italian, Rick...",NaN,[Gil Fronsdal],"[Bristol Universitys Peter Rogers, Drs Chris, ...",NaN,NaN,NaN,NaN
7,2007-01-08,"Photoshop CS3 Way, Photoshop CS3 Public Beta, ...","{'Photoshop CS3 Way': 1, 'Photoshop CS3 Public...","{'North': 2, 'IndieFeed Alternative Modern Roc...","[{0: ['1:13:52', '1:31:17', '2:01:30 Closing',...",0.657359,"[1:13:52, 1:31:17, 2:01:30 Closing, 37:34 Revi...",NaN,NaN,"[Core Security, IndieFeed Alternative Modern R...",NaN,NaN,NaN,NaN
8,2007-01-09,"Italian, Italian, Apple, Steve Jobs, Macworld ...","{'Italian': 8, 'Apple': 1, 'Steve Jobs': 1, 'M...","{'Italian': 8, 'Ben': 3, 'Spanish': 3, 'Guava ...","[{-1: ['Andrew', 'Eric', 'San Francisco', 'iTu...",0.620065,"[Italian, Spain, Spanish]","[Andrew, Eric, San Francisco, iTunes]","[Apple TV, Chicken Soup, Deathly Hallows, Hall...","[Apple, Mac, iPhone]",[Ben],NaN,NaN,NaN
9,2007-01-10,"English, Unnateral Helpers - Gettin', IndieFee...","{'English': 1, 'Unnateral Helpers - Gettin'': ...","{'Guava Duff': 2, 'Italian': 7, 'Spanish': 2, ...","[{1: ['Chris Farmer', 'El Fuego', 'Fed', 'Fede...",0.588905,"[English, Italian, Spanish]",[Patreon],NaN,"[Chris Farmer, El Fuego, Fed, Federal Reserve,...",NaN,NaN,NaN,NaN


In [0]:
# reorder the expanded columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

test = test[['dates','entity','entities_4_days','refined_entities_4_days','groups','silhouettes',-1,0,1,2,3,4,5,6]]
test

,dates,entity,entities_4_days,refined_entities_4_days,groups,silhouettes,-1,0,1,2,3,4,5,6
0,2007-01-01,"Episode, iTunes, New Year, Radio Detective Story Hour, - Murder Clinic, New Year, Murder Clinic, Saturday Morning, Anne Farnsworth, Podsafe Music Network, German, German, New Year Resolutions, The Disney Year, Review, Ricky, Paul Back, Ricky Brigante, Paul Barrie, Disney Year, WindowtotheMagic, Auld Lang Synes, Phil, Dan, regulars, Greg, CJ, Microsoft, Nintendo, Sony, Atari, NESes, Super NESes, Gears of War, Viva Pinata, Gran Turismo, PS3, Zelda:, Pokemon Ruby","{'Episode': 1, 'iTunes': 1, 'New Year': 2, 'Radio Detective Story Hour': 1, '- Murder Clinic': 1, 'Murder Clinic': 1, 'Saturday Morning': 1, 'Anne Farnsworth': 1, 'Podsafe Music Network': 1, 'German': 2, 'New Year Resolutions': 2, 'The Disney Year': 1, 'Review': 1, 'Ricky': 1, 'Paul Back': 1, 'Ricky Brigante': 1, 'Paul Barrie': 1, 'Disney Year': 1, 'WindowtotheMagic': 1, 'Auld Lang Synes': 1, 'Phil': 1, 'Dan': 1, 'regulars': 1, 'Greg': 1, 'CJ': 1, 'Microsoft': 1, 'Nintendo': 1, 'Sony': 1, 'Atari': 1, 'NESes': 1, 'Super NESes': 1, 'Gears of War': 1, 'Viva Pinata': 1, 'Gran Turismo': 1, 'PS3': 1, 'Zelda:': 1, 'Pokemon Ruby': 1, 'Prager': 1, 'Who Really Cares: The suprising': 1, 'Dennis Prager': 1, 'Arthur Brooks': 1, 'Syracuse University': 1, '8MB': 1, 'Dennis Pragers': 1, 'julienne': 1, 'Italian': 2, 'Idiotchild - Mouth Watering': 1, 'IndieFeed Alternative Modern Rock': 1, 'Chloe Day -': 1, 'Chloe Day': 1, 'IndieFeed Indie Pop': 1, 'Caltech Lecture Series': 1, 'Sam Harris': 2, 'Skepticality': 1, 'Symposium on Science': 1, 'Religion & Politics': 1, 'Caltech': 1, 'New York Times': 1, 'The End of Faith': 1, 'Christian': 1, 'Molly Ringwald': 1, 'Sweet Charity': 1, 'Spanish': 2, 'Speeding Bullets': 1, 'DC': 1, 'Raging Bullets of': 1, 'Jim': 1, 'Sean': 1, 'Manhunter Challenge': 1, 'Justice Society of America': 1, 'Brent AKA Knightwingbk': 1, 'Comic Timing Time': 1, '0:00 Opening': 1, '1:17:47 Manhunter Challenge': 1, 'Comic Timings Ian Levenstein': 1, 'John Mayo': 1}","{'New Year': 2, 'German': 2, 'New Year Resolutions': 2, 'Italian': 2, 'Sam Harris': 2, 'Spanish': 2}","[{0: ['- Murder Clinic', 'Anne Farnsworth', 'Atari', 'Auld Lang Synes', 'Disney Year', 'Episode', 'Gears of War', 'Gran Turismo', 'Murder Clinic', 'NESes', 'New Year Resolutions', 'PS3', 'Paul Back', 'Paul Barrie', 'Podsafe Music Network', 'Pokemon Ruby', 'Radio Detective Story Hour', 'Ricky', 'Ricky Brigante', 'Saturday Morning', 'Super NESes', 'The Disney Year', 'Viva Pinata', 'Zelda:', 'regulars'], -1: ['CJ', 'Review', 'WindowtotheMagic', 'iTunes'], 2: ['Dan', 'Greg', 'Microsoft', 'Nintendo', 'Phil', 'Sony'], 1: ['German', 'New Year']}]",0.630648,"[CJ, Review, WindowtotheMagic, iTunes]","[- Murder Clinic, Anne Farnsworth, Atari, Auld Lang Synes, Disney Year, Episode, Gears of War, Gran Turismo, Murder Clinic, NESes, New Year Resolutions, PS3, Paul Back, Paul Barrie, Podsafe Music Network, Pokemon Ruby, Radio Detective Story Hour, Ricky, Ricky Brigante, Saturday Morning, Super NESes, The Disney Year, Viva Pinata, Zelda:, regulars]","[German, New Year]","[Dan, Greg, Microsoft, Nintendo, Phil, Sony]",NaN,NaN,NaN,NaN
1,2007-01-02,"Prager, Who Really Cares: The suprising, Dennis Prager, Arthur Brooks, Syracuse University, 8MB, Dennis Pragers, julienne, Italian, Italian, New Year Resolutions","{'Prager': 1, 'Who Really Cares: The suprising': 1, 'Dennis Prager': 1, 'Arthur Brooks': 1, 'Syracuse University': 1, '8MB': 1, 'Dennis Pragers': 1, 'julienne': 1, 'Italian': 2, 'New Year Resolutions': 1, 'Idiotchild - Mouth Watering': 1, 'IndieFeed Alternative Modern Rock': 2, 'Chloe Day -': 1, 'Chloe Day': 1, 'IndieFeed Indie Pop': 2, 'Caltech Lecture Series': 1, 'Sam Harris': 2, 'Skepticality': 1, 'Symposium on Science': 1, 'Religion & Politics': 1, 'Caltech': 1, 'New York Times': 1, 'The End of Faith': 1, 'Christian': 1, 'Molly Ringwald': 1, 'Sweet Charity': 1, 'Spanish': 4, 'Speeding Bullets': 1, 'DC': 1, 'Raging Bul

In [0]:
from datetime import timedelta
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

def event_duration(df):
    '''
    Takes in the dataframe from "expand_groups_to_event_columns"
    For every two consecutive rows R1 and R2, assuming each row has e0, e1, ... e5 number of events
    Find every pair (R1e0, R2e0), (R1e0, R2e1), ... (R1e5, R2e5) total 25 combinations
    For each combination, call "percent_overlap" to calculate a percentage score
    For each event in R1, take the max(scores), and the R2 argmax(scores) event as its "next_day_event"
    
    Create a dataframe D to keep track of R1 events. D[['features','duration','start_date','end_date']]
    Initialize all start and end dates as the first row date.
    For those that found a "next_day_event", D[e] duration increments by 1, D[e] end date increment by 1.
    Returns D, the event dataframe.
    '''
    
    # initialize the events dataframe
    columns = ['features','duration','start','end','overlap']
    events_df = pd.DataFrame(columns = columns)
    #events_df2 = pd.DataFrame(columns = columns)
    prev_group = np.nan
    
    # Compare events for every two rows in the dataset
    for i in range(len(df)-1):
        
        R1 = df.iloc[i,:]
        R2 = df.iloc[i+1,:]
        m = [] # an n by n matrix, each entry is an overlap score 
        dur = 1
        start = R1['dates']
        end = R1['dates']
        
        # try every pair between events in row1 and events in row2
        for j in range(7):
            mm = [] # a n by 1 vector
            for k in range(7):
                try:
                    OL, OM, OR = percent_overlap(R1[j],R2[k])
                    mm.append(max(OL, OM, OR))
                except TypeError:
                    mm.append(-1)
                    pass
            m.append(mm)
        
        # Find the max overlap between two events in two consecutive time windows
        flat_m = [item for sublist in m for item in sublist]
        max_score = max(flat_m)
        m_arr = np.array(m)
        
        # Search for max overlap percentage entries in m, match events by m index numbers
        if max_score != 0:
            indices = list(zip(*np.where(m_arr == max_score)))
            
            # continuous event
            if indices[0][0] == prev_group: 
                # last event duration + 1, end date + 1, features combine
                fea = events_df.iloc[len(events_df)-1,0] + R2[indices[0][1]] 
                dur = events_df.iloc[len(events_df)-1,1] + 1 
                start = events_df.iloc[len(events_df)-1,2]
                end = timedelta(days=1) + events_df.iloc[len(events_df)-1,3] 
                ove = (events_df.iloc[len(events_df)-1,4] + max_score)/2 
                newdata = [[fea, dur, start, end, ove]]
                events_df = events_df[:-1]
                events_df = events_df.append(pd.DataFrame(newdata, columns = columns), ignore_index=True)
            
            # new event
            else: 
                # create new features, end date + 1, duration + 1
                fea = R1[indices[0][0]]+R2[indices[0][1]]
                dur = dur + 1
                end = R2['dates']
                newdata = [[fea, dur, start, end, max_score]]
                events_df = events_df.append(pd.DataFrame(newdata, columns = columns), ignore_index=True)
            prev_group = indices[0][1]
        
        # For those zero overlaps, count them as individual events and append to the events dataframe
        else:
            ind = set(np.where(m_arr == 0)[0])
            print(i,ind)
            for e in ind:
                feat = df[e][i]
                start = df['dates'][i]
                end = df['dates'][i]
                dur = 1
                newd = [[feat, dur, start, end, 0]]
                events_df = events_df.append(pd.DataFrame(newd, columns = columns), ignore_index=True)

    return events_df#, events_df2
        
       
testout = event_duration(test)
testout

1 {0, 1}
2 {0, 1, 2, 3}
3 {0, 1}
4 {0, 1}
5 {0, 1}
11 {0, 1}
12 {0}
13 {0, 1, 2}
14 {0, 1}
15 {0, 1}
16 {0, 1}
18 {0, 1, 2, 3}
19 {0, 1}
20 {0, 1, 2, 3, 4, 5, 6}
21 {0, 1, 2, 3, 4, 5, 6}
22 {0, 1, 2}
23 {0}
25 {0, 1}
26 {0}
27 {0, 1, 2}
28 {0, 1}
29 {0}
30 {0, 1, 2}
31 {0, 1}
32 {0, 1, 2}
33 {0, 1, 2, 3}
36 {0, 1, 2}
37 {0, 1}
38 {0, 1, 2, 3}
40 {0, 1, 2}
46 {0, 1, 2, 3, 4, 5}
47 {0, 1}


,features,duration,start,end,overlap
0,"[- Murder Clinic, Anne Farnsworth, Atari, Auld Lang Synes, Disney Year, Episode, Gears of War, Gran Turismo, Murder Clinic, NESes, New Year Resolutions, PS3, Paul Back, Paul Barrie, Podsafe Music Network, Pokemon Ruby, Radio Detective Story Hour, Ricky, Ricky Brigante, Saturday Morning, Super NESes, The Disney Year, Viva Pinata, Zelda:, regulars, 8MB, Arthur Brooks, Dennis Pragers, New Year Resolutions, Prager, Syracuse University, Who Really Cares: The suprising, julienne]",2,2007-01-01,2007-01-02,12.500000
1,"[8MB, Arthur Brooks, Dennis Pragers, New Year Resolutions, Prager, Syracuse University, Who Really Cares: The suprising, julienne]",1,2007-01-02,2007-01-02,0.000000
2,[Italian],1,2007-01-02,2007-01-02,0.000000
3,"[Caltech, Caltech Lecture Series, Chloe Day, Chloe Day -, Idiotchild - Mouth Watering, Molly Ringwald, Religion & Politics, Sweet Charity, Symposium on Science, The End of Faith]",1,2007-01-03,2007-01-03,0.000000
4,"[IndieFeed Alternative Modern Rock, IndieFeed Indie Pop, Skepticality]",1,2007-01-03,2007-01-03,0.000000
5,[Sam Harris],1,2007-01-03,2007-01-03,0.000000
6,[Spanish],1,2007-01-03,2007-01-03,0.000000
7,"[0:00 Opening, 1:17:47 Manhunter Challenge, Brent AKA Knightwingbk, Comic Timing Time, Comic Timings Ian Levenstein, John Mayo, Justice Society of America, Manhunter Challenge, Raging Bullets of, Speeding Bullets]",1,2007-01-04,2007-01-04,0.000000
8,"[DC, Jim, Sean]",1,2007-01-04,2007-01-04,0.000000
9,"[AP, AZ, American, Ben, Celtfather, Celtic, Celtic Music, Celtic Music Magazine, Celtic Music Podcast, Cindy, Dublin, French, IndieFeed Alternative Modern Rock, IndieFeed Indie Pop, Ireland, Irish, Irish & Celtic Music Podcast, Jason, Marc Gunn, Mary, Sam, Spanish, Sun, Tom, iTunes]",1,2007-01-05,2007-01-05,0.000000


### Step 3: Assign scores

In [0]:
### Step 3: Assign scores
def is_event_feature(df):
    sentences = get_all_unique_entities()
    all_entities = [item for sublist in sentences for item in sublist]
    wide_df, not_features_flat = is_event_feature(df,'groups')
    features = all_entities - not_features_flat
    return features

def sort_features():
    '''
    Given a list of dictionaries with unique keys as features, and values as frequencies 
    (output from CountFrequency)
    Return the sorted list of dictionaries
    '''
    sentences2 = get_sentences_2(df_window)
    all_features = [item for sublist in sentences2 for item in sublist]
    all_features.sort()
    return all_features

def feature_score(sorted_features):
    '''
    Loop through each item (a dictionary) in the list
    value = value + weight if is feature
    return sorted_features
    '''

def event_score(events_df):
    '''
    Takes in the events_df returned by "event_duration"
    flatten it so that each row is a feature and its duration
    return the dictionary as "duration"
    '''

def final_score(sorted_features, duration):
    '''
    Takes the sorted_features output from "feature_score"
    for each key in sorted_features, find it the duration dictionary, multiple its value by the duration's value
    return sorted_features as the final score
    '''
    f = features_score(df)
    e = event_score(events_df)
    return f * e
    
    
def get_all_unique_entities():
    m = [df_window['entity'][i] for i in range(len(df_window['entity']))]
    sentences = []
    for item in m:
        sentences.append(item.split(", "))
    return sentences
    
def not_event_feature(df, col):
    '''
    Takes in a dataframe, expands the grouped dictionary column horizontally,
    Returns a list of entities that are not event features, and the wide dataframe 
    '''
    df2 = json_normalize(df[col])
    wide_df = pd.concat([df, df2], axis=1)
    not_features = list(wide_df[-1]) # dbscan has decided is noise with group label -1
    not_features_flat = [item for sublist in not_features for item in sublist]
    return wide_df, not_features_flat